In [4]:
import os
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from PIL import Image
from torchvision import transforms, models
import torch.nn.functional as F

# Vérifier si CUDA est disponible, sinon utiliser le CPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)
# Architecture simplifiée du CNN
class MelanomaCNN(nn.Module):
    def __init__(self):
        super(MelanomaCNN, self).__init__()
        self.conv = nn.Conv2d(3, 16, kernel_size=3, padding=1)
        self.fc = nn.Linear(16 * 112 * 112, 1)

    def forward(self, x):
        x = F.relu(self.conv(x))
        x = F.max_pool2d(x, 2)
        x = x.view(-1, 16 * 112 * 112)
        x = torch.sigmoid(self.fc(x))
        return x

class MelanomaTestDataset(Dataset):
    def __init__(self, img_dir, transform=None, limit=None):
        self.img_names = os.listdir(img_dir)[:limit]  # Ajoutez la limitation ici
        self.img_dir = img_dir
        self.transform = transform

    def __len__(self):
        return len(self.img_names)

    def __getitem__(self, idx):
        img_name = os.path.join(self.img_dir, self.img_names[idx])
        image = Image.open(img_name)
        if self.transform:
            image = self.transform(image)
        return image, self.img_names[idx]

cuda


In [5]:
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

# Charger ResNet50 (sans les poids pré-entraînés pour le moment)
model = models.resnet101(pretrained=False)

# Remplacer la dernière couche fc pour notre classification binaire
model.fc = nn.Sequential(
    nn.Linear(2048, 1),
    nn.Sigmoid()
)

# Charger les poids du modèle fine-tuné
model.load_state_dict(torch.load('melanoma_model.pth'))  # Mettez à jour le chemin
model.eval()

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

test_dataset = MelanomaTestDataset(img_dir='../../data/test-resized/test-resized', transform=transform)  # Mettez à jour le chemin
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

c:\Users\bapti\AppData\Local\Programs\Python\Python310\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\bapti\AppData\Local\Programs\Python\Python310\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


In [ ]:
# Effectuer des prédictions
predictions = []
image_names = []

with torch.no_grad():
    for data, names in test_loader:
        data = data.to(device)
        outputs = model(data).squeeze()
        # Convertir les sorties tensor en liste de valeurs flottantes
        predictions.extend(outputs.tolist())
        # Retirer l'extension des noms d'image
        image_names.extend([n.split('.')[0] for n in names])

# Générer un fichier CSV pour la soumission
submission_df = pd.DataFrame({
    'image_name': image_names,
    'target': predictions
})

submission_df.to_csv('submission_resnet50.csv', index=False)